<h1>Challenge 5: The Traveling Salesman Problem</h1>

Solve a coding challenge on the famous traveling salesman problem.

> **Problem statement**

You are given a map that has nn cities on it. All of the cities are connected directly to each other via distinct routes of variable lengths. You, being a salesman, have to travel to each of these cities on a business trip. But your company wants you to be very careful with the travel expenses and wants you to spend the least amount possible. Expenditure is a function of the distance traveled, so you want to minimize the total distance traveled. Find the length of the shortest path to travel all the cities.

> **Input**

You will be given a 2-d matrix, distances, of size n \times nn×n denoting the distance between all the cities. To check the distance from i^(th) city to j^(th) city, you can simply do distances[i][j]. The matrix does not necessarily have to be symmetrical, i.e., distances[i][j] ≠ distances[j][i]. You can start from any city to travel around all the other cities until you come back to the same city you started traveling from. You should not visit any city twice other than the first city.

distances = [

      [0, 10, 20],

      [12, 0, 10],

      [19, 11, 0]

]

> **Output**

Your algorithm should return the minimum distance traveled to traverse every city; each city other than the first. The chosen city should only be traveled to once.

TSP(distances) = 39

> **Coding challenge**

This is a difficult problem, so let’s start with a very basic example. Think about how many paths exist and how you can find the most optimal one amongst them. Once you have written a working algorithm, optimize it using dynamic programming.

> **Solution #1: Simple recursion**




In [1]:
import numpy as np

def TSPrecursive(distances, check, index, start):
  minimum = np.inf
  for i in range(len(distances)):
    if i != index and i != start and i not in check:
      check[i] = 1
      minimum = min(minimum, distances[index][i]+TSPrecursive(distances, check, i, start))
      del check[i]
  if minimum == np.inf:
    return distances[index][start]
  return minimum

def TSP(distances):
  check = {}
  minimum = np.inf
  for i in range(len(distances)):
    minimum = min(minimum, TSPrecursive(distances, check, i, i))
  return minimum

print(TSP([
      [0, 10, 20],
      [12, 0, 10],
      [19, 11, 0],
]))

39


> **Solution #2: Top-down dynamic programming**




Let’s see how this problem satisfies both pre-requisites of using dynamic programming.

**Optimal substructure**

This problem can be best explained using some concepts of set theory. As the order in which cities are visited is not important as long as all the cities are visited, unordered sets are a perfect fit for the problem. To construct a set of size nn, or to find an optimal answer to the problem of n cities, we can explore which subproblems of size n-1 leads to the optimal solution. Thus, if we have optimal answers to all the subproblems of size n-1, we can construct an optimal answer to the main problem of n.

**Overlapping subproblem**

Since we are finding permutations, we should expect to find a number of repetitions in the calculation of these permutations.

In [3]:
import numpy as np

def TSPrecursive(distances, check, index, end, memo):
  keys = tuple(sorted(check.keys()))
  if (keys, index) in memo:
    return memo[(keys, index)]
  minimum = np.inf
  for i in range(len(distances)):
    if i != index and i != end and i not in check:
      check[i] = 1
      minimum = min(minimum, distances[index][i]+TSPrecursive(distances, check, i, end, memo))
      del check[i]
  if minimum == np.inf:
    return distances[index][end]
  memo[(keys, index)] = minimum
  return memo[(keys, index)]

def TSP(distances):
  check = {}
  minimum = np.inf
  for i in range(len(distances)):
    minimum = min(minimum, TSPrecursive(distances, check, i, i, {}))
  return minimum

print(TSP([
      [0, 10, 20],
      [12, 0, 10],
      [19, 11, 0],
]))

39


>**Solution #3: Bottom-up dynamic programming**




In [4]:
import numpy as np

def findSubsets(numbers, i, subsets):
	if len(numbers) == i:
		return subsets
	if len(subsets) == 0:
		return findSubsets(numbers, i+1, [(), tuple([numbers[i]])])
	temp_subsets = []
	for subset in subsets:
		temp_subsets += [tuple(list(subset) + [numbers[i]])]
	return findSubsets(numbers, i+1, subsets + temp_subsets)
  
# function to find shortest path starting from city `start` and back to it
def TSPbottomup(distances, start):
  dp = {} # dp table
  # subproblem of travelling to second city from start city
  for i in range(len(distances)):
    dp[(tuple([i]), i)] = distances[start][i]
  # find all possible subsets of the cities
  subsets = findSubsets(list(range(len(distances))), 0, [])
  # solve for subset of each size from 2 to n
  for subsetSize in range(2,len(distances)+1):
    for subset in subsets:
      if len(subset) == subsetSize:
        # evaluating minimum cost to travel `subsetSize` number of cities while ending up at each city 
        for lastCity in subset:
          dp[(subset, lastCity)] = np.inf
          l = list(subset)
          l.remove(lastCity)
          subset2 = tuple(l)
          # to end up at city given by `lastCity`, it should be the last city to be traveled
          for city in subset2:
            dp[(subset, lastCity)] = min(dp[(subset, lastCity)], dp[(subset2, city)] + distances[city][lastCity])
  # return answer to the problem of travlling all cities while ending up at start city
  return dp[(subsets[-1], start)]

def TSP(distances):
  minimum = np.inf
  for i in range(len(distances)):
    minimum = min(minimum, TSPbottomup(distances, i))
  return minimum

print(TSP([
      [0, 10, 20],
      [12, 0, 10],
      [19, 11, 0],
]))

39
